# Processing data for manual time lapse inversion
> Framework
- import baseline & survey
- process reciprocal error
- save into .protocol file
- gather baseline + survey for timelapse inversion


>> check difference inversion to define if we need baseline modeling

In [57]:
# Import functions

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.dates as mdates
from os.path import isfile, join, exists
from resipy import Project #to install: pip install resipy

# New function

def ER_filtering(data, DevTh, Pot, Curr, Stk, AppR):
    # Calculate resistance
    data['Rho (Ohm.m)'] = data['VMN (mV)'] / data['IAB (mA)']

    # Filtering in function of deviation
    idx = data['Dev. Rho (%)'] > DevTh
    data['Rho (Ohm.m)'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from stacking dev error threshold')

    # Filtering in function of potential
    idx = data['VMN (mV)'] == Pot
    data['Rho (Ohm.m)'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from negative VMN')

    # Filtering in function of current
    idx = data['IAB (mA)'] <= Curr
    data['Rho (Ohm.m)'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from negative IAB')

    # Filtering in function of stacking
    idx = data['IAB (mA)'] <= Stk
    data['Rho (Ohm.m)'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from stacking')

    # Filtering in function of app. resistivity
    idx = data['IAB (mA)'] >= AppR
    data['Rho (Ohm.m)'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from too high resistivity')

    return data

def saveProtocol(Stdata,Rcdata,sfile):
    d1 = np.column_stack(
        (Stdata['N° electrode A'],
         Stdata['N° electrode B'], 
         Stdata['N° electrode M'],
         Stdata['N° electrode N'], 
         Stdata['Rho (Ohm.m)']))
    d2 = np.column_stack(
        (Rcdata['N° electrode A'],
         Rcdata['N° electrode B'], 
         Rcdata['N° electrode M'],
         Rcdata['N° electrode N'], 
         Rcdata['Rho (Ohm.m)']))
    
    data = np.concatenate((d1,d2), axis=0)

    with open(sfile, 'wt') as f:  
        f.write("Spa.1 Spa.2 Spa.3 Spa.4 Rho \n")
        for row in data:
            f.write('\t'.join(map(str, row)) + '\n')
    

> Data import

> Data filtering
- Deviation threshold: DevTh (in %)
- Positive VMN potential: Pot (in mV)
- Postive IAB current: Curr(in mA)
- More than X stack: Stk (nb)
- With App. Resistivity lower than Y: AppR (Ohm.m)

> Save

In [58]:
# Import data

date = ['2406', '2407', '2408']
Line = 'A'

Dtfolder = 'C:/Users/Marc/OneDrive - Colorado School of Mines/Documents/4-Colorado/2-GordonGulch/0-Data/24_ERdata/2-Txtfiles/'
Svfolder = 'C:/Users/Marc/OneDrive - Colorado School of Mines/Documents/4-Colorado/2-GordonGulch/0-Data/24_ERdata/24-Line'

# Data processing

DevTh = 10
Pot = 0
Curr = 0
Stk = 1
AppR = 999999 #use 999999 to not include this.

for d in date:
    print(d)
    # standard data
    file = os.path.join(Dtfolder + d +'-ER' + Line + '_stand.txt')
    Dst = pd.read_csv(file, delimiter=' ', encoding='latin-1')
    # reciprocal data
    file = os.path.join(Dtfolder + d +'-ER' + Line + '_recip.txt')
    Drp = pd.read_csv(file, delimiter=' ', encoding='latin-1')

    Dst = ER_filtering(Dst, DevTh, Pot, Curr, Stk, AppR)
    Drp = ER_filtering(Drp, DevTh, Pot, Curr, Stk, AppR)

    # Save file
    sfile = os.path.join(Svfolder + Line + '/' + d + '-ER.dat')
    saveProtocol(Dst,Drp,sfile)


2406
Filtered out 41 from stacking dev error threshold
Filtered out 1 from negative VMN
Filtered out 0 from negative IAB
Filtered out 0 from stacking
Filtered out 0 from too high resistivity
Filtered out 2 from stacking dev error threshold
Filtered out 0 from negative VMN
Filtered out 0 from negative IAB
Filtered out 0 from stacking
Filtered out 0 from too high resistivity
2407
Filtered out 279 from stacking dev error threshold
Filtered out 176 from negative VMN
Filtered out 176 from negative IAB
Filtered out 222 from stacking
Filtered out 0 from too high resistivity
Filtered out 34 from stacking dev error threshold
Filtered out 0 from negative VMN
Filtered out 0 from negative IAB
Filtered out 2 from stacking
Filtered out 0 from too high resistivity
2408
Filtered out 322 from stacking dev error threshold
Filtered out 230 from negative VMN
Filtered out 228 from negative IAB
Filtered out 282 from stacking
Filtered out 0 from too high resistivity
Filtered out 56 from stacking dev error th

C:\Users\Marc\AppData\Local\Temp\ipykernel_10884\1913253658.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rho (Ohm.m)'][idx] = np.nan
C:\Users\Marc\AppData\Local\Temp\ipykernel_10884\1913253658.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rho (Ohm.m)'][idx] = np.nan
C:\Users\Marc\AppData\Local\Temp\ipykernel_10884\1913253658.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rho (Ohm.m)'][idx] = np.na